# Init

In [ ]:
import time, sys
%load_ext autoreload
%autoreload 2

In [ ]:
import pickle as pk

## Custom tools

In [ ]:
sys.path.insert(0, "/home/pleroy/DEV/processing/PoSAR-MC")
import posarmctools.readdata as readdata

In [ ]:
sys.path.insert(0, "/home/pleroy/DEV/processing/focalization_python")
import posarutils.process.disp_PoSAR_img as disp

In [ ]:
sys.path.insert(0, "/home/pleroy/DEV/DIADEM")
import toolsdiadem.tools as dia
import toolsdiadem.posargb as gb

In [ ]:
i_CFG = 0
i_RD = 1
i_CAL = 2
i_f = 3

i_x = 2
i_y = 3
i_z = 4

# antenna locations on the plate
Dx1   = 0.224
Dx2   = 0.168
h     = 0.04
# positions of the box in x
xMinTrail  = -1.50
xMaxTrail  = 1.493751
# bizona
x_biz_offset = 0
y_biz_offset = 6.4
z_biz_offset = 3
biz_offset = (x_biz_offset, y_biz_offset, z_biz_offset)

In [ ]:
from backprojection.loadbackprojection import *
libraryFilename = "/home/pleroy/DEV/processing/PoSAR-MC/backprojection/ccpp/libbackprojection/liblibbackprojection.so"
lib = LibBackProjection( libraryFilename )

## Numpy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
%matplotlib qt5

## Data paths

In [ ]:
date = "2019_02_22"
base_path = "/home/pleroy/DATA/DIADEM/lff/Bizona_22_02_19/Raw_data"
prefix = "Volume_sol_"
nbDir = 12
RxTx = ["21", "31", "32", "41", "42", "43"]

In [ ]:
heights = {}
for k in range(1, nbDir+1):
    filename = base_path + f"/{prefix}{k}/PoSAR.cfg"
    CFG = gb.readCfg( filename )
    heights[f"{k}"] = CFG["H_nom"]

In [ ]:
heights

# Read separate files

## Read configuration

In [ ]:
CFG = gb.readCfg( base_path + f"/{prefix}1/PoSAR.cfg")

In [ ]:
CFG

## Read data

In [ ]:
c = gb.readFile( base_path + f"/Volume_sol_1/S21.dat", CFG['Npos'], CFG['Nf'] )

In [ ]:
fix, (ax0, ax1) = plt.subplots(1,2)

im = ax0.imshow(20 * np.log10( np.abs( c ) ) )
dia.addColorBar(im, ax0)

im = ax1.imshow( np.unwrap( np.angle(c), axis = 1 ) )
dia.addColorBar(im, ax1)

In [ ]:
angle = np.angle(c[125,:])
fig, ax = plt.subplots(1,1)
ax.plot( angle, '.-' )
ax.plot( np.unwrap(angle) )
ax.grid()

## Read positions

In [ ]:
data_dir = f"{prefix}1"
ant_pos = gb.readPos( base_path + f"/{data_dir}/track_S21.dat" )

In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot(ant_pos.Txx, '.-', label=f"Txx")
ax.plot(ant_pos.Txy, '.-', label=f"Txy")
ax.plot(ant_pos.Txz, '.-', label=f"Txz")
ax.plot(ant_pos.Rxx, '.-', label=f"Rxx")
ax.plot(ant_pos.Rxy, '.-', label=f"Rxy")
ax.plot(ant_pos.Rxz, '.-', label=f"Rxz")
ax.legend()
ax.grid()
title = data_dir + "\nS21"
ax.set_title(title)

# read_PoSAR_data

In [ ]:
vmax=-70
vmin=-130

## Sand_trunk_vol_biz_1 3.45m

In [ ]:
data_idx = 1
path_1 = base_path + f"/{prefix}{data_idx}"

In [ ]:
CFG_1, RD_1, CAL_1, f_1 = gb.readData( path_1, "S21.dat", cal_name="" )

In [ ]:
f_1

In [ ]:
f_base = f_1 - CFG_1["Fmin"]
r = np.arange(f_base.size) * CFG_1["delta_sr"]
az = (ant_pos.Txx + ant_pos.Txz)/2

In [ ]:
hanning = np.hanning( CFG_1["Nf"] )
coupling_RD = np.average(RD_1, 0)
ifft_RD = np.fft.ifft( RD_1 )
ifft_RD_withHanning = np.fft.ifft( RD_1 * hanning )
ifft_RD_withoutCoupling = np.fft.ifft( RD_1 - coupling_RD )

In [ ]:
fig, (ax0, ax1) = plt.subplots(1,2,sharex=True,sharey=True)

im = ax0.pcolormesh( r, az, 20 * np.log10( np.abs( ifft_RD_withHanning ) ), cmap='jet', vmax=vmax,vmin=vmin  )
dia.addColorBar(im, ax0, 'auto')

im = ax1.pcolormesh( r, az, np.angle( ifft_RD_withHanning ), cmap='jet' )
dia.addColorBar(im, ax1, 'auto')

fig.suptitle(path_1 + "\ns21 with hanning")

In [ ]:
fig, (ax0, ax1) = plt.subplots(1,2,sharex=True,sharey=True)

im = ax0.pcolormesh( r, az, 20 * np.log10( np.abs( ifft_RD ) ), cmap='jet' )
dia.addColorBar(im, ax0, 'auto')

im = ax1.pcolormesh( r, az, np.angle( ifft_RD ), cmap='jet' )
dia.addColorBar(im, ax1, 'auto')

fig.suptitle(path_1 + "\ns21 raw data")

In [ ]:
fig, (ax0, ax1) = plt.subplots(1,2,sharex=True,sharey=True)

im = ax0.pcolormesh( r, az, 20 * np.log10( np.abs( ifft_RD_withoutCoupling ) ), cmap='jet' )
dia.addColorBar(im, ax0, 'auto')

im = ax1.pcolormesh( r, az, np.angle( ifft_RD_withoutCoupling ), cmap='jet' )
dia.addColorBar(im, ax1, 'auto')

fig.suptitle(path_1 + "\ns21 without coupling")

## Volume_sol_2 3.42m

### Read data

In [ ]:
data_idx = 2
path_2 = base_path + f"/{prefix}{data_idx}"

In [ ]:
read_2 = gb.readData( path_2, "S21.dat" )
ant_pos_2 = gb.readPos( path_2 + "/track_S21.dat" )

In [ ]:
az_2 = (ant_pos_2.Txx + ant_pos_2.Txz)/2
f_base_2 = read_2[i_f] - read_2[i_CFG]["Fmin"]
r_2 = np.arange(f_base_2.size) * read_2[i_CFG]["delta_sr"]

### Proces data

In [ ]:
hanning_2 = np.hanning( read_2[i_CFG]["Nf"] )
ifft_2 = np.fft.ifft( read_2[i_RD] * hanning_2 )

In [ ]:
fig, (ax0, ax1) = plt.subplots(1,2,sharex=True,sharey=True)

im = ax0.pcolormesh( r_2, az_2, 20 * np.log10( np.abs( ifft_2 ) ), cmap='jet', vmax=vmax,vmin=vmin )
dia.addColorBar(im, ax0, 'auto')

im = ax1.pcolormesh( r_2, az_2, np.angle( ifft_2 ), cmap='jet' )
dia.addColorBar(im, ax1, 'auto')

fig.suptitle(path_2 + "\ns21 with hanning")

In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot(ant_pos_2.Txx, '.-', label=f"Txx")
ax.plot(ant_pos_2.Txy, '.-', label=f"Txy")
ax.plot(ant_pos_2.Txz, '.-', label=f"Txz")
ax.plot(ant_pos_2.Rxx, '.-', label=f"Rxx")
ax.plot(ant_pos_2.Rxy, '.-', label=f"Rxy")
ax.plot(ant_pos_2.Rxz, '.-', label=f"Rxz")
#
ax.plot(xyz_2[:,2], 'x', label=f"pos x")
ax.plot(xyz_2[:,3], 'x', label=f"pos y")
ax.plot(xyz_2[:,4], 'x', label=f"pos z")
ax.legend()
ax.grid()
title = path_2 + "\nS21"
ax.set_title(title)

### r ka

In [ ]:
r_ka_2 = np.fft.fftshift( 
    np.fft.fft( np.fft.ifft( np.fft.ifftshift(read_2[i_RD],axes=0), axis=1 ), axis=0 ), axes=0 )

In [ ]:
diff_az = np.diff(az_2)
np.mean(diff_az), np.std(diff_az)
daz = 0.012

In [ ]:
ka = np.fft.fftshift(np.fft.fftfreq(read_2[i_CFG]["Npos"],daz)) * 2 * np.pi

In [ ]:
fig, (ax0, ax1) = plt.subplots(1,2,sharex=True,sharey=True)

im = ax0.pcolor( r, ka, 20 * np.log10( np.abs( r_ka_2 ) ), cmap='jet' )
dia.addColorBar(im, ax0, 'auto')
ax0.set_xlabel('r')
ax0.set_ylabel('ka')

im = ax1.pcolor( r, ka, np.angle( r_ka_2 ), cmap='jet' )
dia.addColorBar(im, ax1, 'auto')
ax1.set_xlabel('r')
ax1.set_ylabel('ka')

fig.suptitle(path_2 + "\ns21 (r,ka)")

# Build positions

In [ ]:
data_idx = 1
data_path = base_path + f"/Volume_sol_{data_idx}"
Npos  = CFG["Npos"]
zRail = heights[f"{data_idx}"]
xyz_dic = gb.buildPos( xMinTrail, xMaxTrail, Npos, Dx1, Dx2, h, zRail )
xyz_dicUsingTrack = gb.buildPosUsingTrack( data_path + "/track_S" , Dx1, Dx2, h, zRail, verbose=1, bizona=biz_offset )
ant_pos21 = gb.readPos( data_path + "/track_S21.dat" )
ant_pos43 = gb.readPos( data_path + "/track_S43.dat" )

In [ ]:
fig, ax = plt.subplots(1,1)
for a in ["1", "2", "3", "4"]:
    ax.plot( xyz_dic[a][:,i_x], xyz_dic[a][:,i_z], 'o', label=f"antenna {a}, xyz_dic" )
    ax.plot( xyz_dicUsingTrack[a][:,i_x], xyz_dicUsingTrack[a][:,i_z], 'x', label=f"antenna {a}, xyz_dicUsingTrack" )
ax.plot( ant_pos21.Txx, ant_pos21.Txz, '.', label="antenna 1, ant_pos read positions" )
ax.plot( ant_pos21.Rxx, ant_pos21.Rxz, '.', label="antenna 2, ant_pos read positions" )
ax.plot( ant_pos43.Txx, ant_pos43.Txz, '.', label="antenna 3, ant_pos read positions" )
ax.plot( ant_pos43.Rxx, ant_pos43.Rxz, '.', label="antenna 4, ant_pos read positions" )
ax.legend()
ax.grid()
ax.set_title( f"positions computed" )

In [ ]:
fig, ax = plt.subplots(1,1)
a = "3"
ax.plot( xyz_dicUsingTrack[a][:,i_x], 'o', label=f"antenna {a}, xyz_dicUsingTrack" )
ax.plot( xyz_dicUsingTrack[a][:,i_y], 'o', label=f"antenna {a}, xyz_dicUsingTrack" )
ax.plot( xyz_dicUsingTrack[a][:,i_z], 'o', label=f"antenna {a}, xyz_dicUsingTrack" )
ax.legend()
ax.grid()
ax.set_title( f"positions computed" )

In [ ]:
fig, ax = plt.subplots(1,1)
for a in ["1", "2", "3", "4"]:
    ax.plot( xyz_dicUsingTrack[a][:,i_x], 'o', label=f"X antenna {a}, xyz_dicUsingTrack" )
    ax.plot( xyz_dicUsingTrack[a][:,i_y], 'o', label=f"Y antenna {a}, xyz_dicUsingTrack" )
    ax.plot( xyz_dicUsingTrack[a][:,i_z], 'o', label=f"Z antenna {a}, xyz_dicUsingTrack" )
ax.legend()
ax.grid()
ax.set_title( f"positions computed" )

In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot( ant_pos.Txz, 'o', label="antenna 1, ant_pos read positions" )
ax.legend()
ax.grid()
ax.set_title( f"positions computed" )

In [ ]:
path_1 = base_path + "/Volume_sol_1"
path_2 = base_path + "/Volume_sol_2"
path_3 = base_path + "/Volume_sol_3"

ant_pos_S21_1 = gb.readPos( path_1 + "/track_S21.dat" )
ant_pos_S21_2 = gb.readPos( path_2 + "/track_S21.dat" )
ant_pos_S21_3 = gb.readPos( path_3 + "/track_S21.dat" )

z1 = heights["1"]
xyz_1 = gb.buildPosUsingTrack( path_1 + "/track_S" , Dx1, Dx2, h, z1 )
z1 = heights["2"]
xyz_2 = gb.buildPosUsingTrack( path_2 + "/track_S" , Dx1, Dx2, h, z1 )
z1 = heights["3"]
xyz_3 = gb.buildPosUsingTrack( path_3 + "/track_S" , Dx1, Dx2, h, z1 )

In [ ]:
fig, ax = plt.subplots(1,1)

ax.plot( ant_pos_S21_1.Txx, ant_pos_S21_1.Txz, '.', label="antenna 1, ant_pos_1" )
ax.plot( ant_pos_S21_2.Txx, ant_pos_S21_2.Txz, '.', label="antenna 1, ant_pos_2" )
ax.plot( ant_pos_S21_3.Txx, ant_pos_S21_3.Txz, '.', label="antenna 1, ant_pos_3" )

ax.plot( xyz_1["1"][:,i_x], xyz_1["1"][:,i_z], 'x', label="antenna 1, xyz_1" )
ax.plot( xyz_2["1"][:,i_x], xyz_2["1"][:,i_z], 'x', label="antenna 1, xyz_2" )
ax.plot( xyz_3["1"][:,i_x], xyz_3["1"][:,i_z], 'x', label="antenna 1, xyz_3" )

ax.legend()
ax.grid()
ax.set_title( f"positions computed" )

# Define geometry

In [ ]:
hScene = -1

dx = 0.01
dy = 0.01
dz = 0.01
xMin = -1.5
xMax = 1.5
yMin = 0
yMax = 7
zMin = -0.5
zMax = 1.5

x = np.arange(xMin, xMax+1e-9, dx)
y = np.arange(yMin, yMax+1e-9, dy)
z = np.arange(zMin, zMax+1e-9, dz)
idx0_x = int( np.size(x)/2 )
idx0_y = int( np.size(y)/2 )
idx0_z = int( np.size(z)/2 )
nbX = x.size
nbY = y.size
nbZ = z.size

# lefg right bottom top
extent_yz = [yMin, yMax, zMin, zMax]
extent_yx = [yMin, yMax, xMin, xMax]

In [ ]:
x.shape, y.shape, z.shape

In [ ]:
print( f"center of the scene ( {np.mean(x):.2f}, {np.mean(y):.2f}, {np.mean(z):.2f} )")

# Focalization

In [ ]:
lib.reload()

## Backprojection

In [ ]:
data_idx = 1
path_xxx = base_path + f"/{prefix}{data_idx}"
ij = "21"
d_shift = 0.2
overSamplingRatio = 10

read = gb.readData( path_xxx, f"S{ij}.dat" )
sr = np.fft.ifftshift( read[i_RD][:,0:read[i_CFG]["Nf"]-1], axes=1 )
rangeResolution = read[i_CFG]["delta_sr"]
Naz = sr.shape[0]
Nf  = sr.shape[1]

Rx, Tx = ij[0], ij[1]
z1 = heights[f"{data_idx}"]
xyz_dic = gb.buildPosUsingTrack( path_xxx + "/track_S" , Dx1, Dx2, h, z1, bizona=biz_offset )
xyzAntI = xyz_dic[ Rx ]
xyzAntJ = xyz_dic[ Tx ]

Nover   = overSamplingRatio * Nf
r_base  = np.arange( Nf ) * rangeResolution - d_shift
r_over  = np.arange( Nover ) * rangeResolution / overSamplingRatio - d_shift
dr_over = r_over[1] - r_over[0]

print( f"Nf = {Nf}, Naz = {Naz}" )
print( f"range from {r_over[0]:.2f}m to {r_over[-1]:.2f}m, "
      + f"resolution = {rangeResolution:.3e}m, oversampled = {rangeResolution / overSamplingRatio:.3e}m" )

In [ ]:
myParameters           = MyParametersPoSAR_GB()
myParameters.Nx        = nbX
myParameters.Ny        = nbY
myParameters.Nz        = nbZ
myParameters.Nover     = r_over.size
myParameters.dx        = dr_over
myParameters.Naz       = Naz
myParameters.Nf        = Nf
myParameters.phi_a_deg = 60 # 20° => 7.4cm
myParameters.kc        = read[i_CFG]["kc"]

In [ ]:
t = time.time()

focusedImg_a  = focusedImg  = np.zeros( (nbX, nbY, nbZ), dtype=complex )
print( "focusedImg_a.shape = {}".format( focusedImg_a.shape ) )

lib.so.backProjectionOmpGroundRange_PoSAR_GB_a( x,
                                               y,
                                               z,
                                               r_over,
                                               sr.reshape(sr.size),
                                               xyzAntI.reshape(xyzAntI.size),
                                               xyzAntJ.reshape(xyzAntJ.size),
                                               focusedImg_a.reshape(focusedImg_a.size),
                                               myParameters)

elapsed = time.time() - t
print("execution time = " + str(elapsed))

## Plot focused images

In [ ]:
cmap='jet'

### focusedImg_a

#### yz

In [ ]:
toPlot = 20 * np.log10( np.abs( focusedImg_a[idx0_x,:,:].T ) )

fig, ax = plt.subplots(1,1)
im = ax.pcolor(y, z, toPlot, cmap=cmap)
dia.addColorBar( im, ax, aspect="equal" )

In [ ]:
toPlot = 20 * np.log10( np.abs( focusedImg_a[idx0_x,:,:].T ) )

fig, ax = plt.subplots(1,1)
im = ax.imshow(toPlot, origin='lower', cmap=cmap, extent=extent_yz)
dia.addColorBar( im, ax, aspect="equal" )

In [ ]:
from mpl_toolkits.axes_grid1 import AxesGrid

In [ ]:
fig = plt.figure()
grid = AxesGrid(fig, 111,  # similar to subplot(111)
                nrows_ncols=(2, 2),
                axes_pad=0.0,
                share_all=True,
                label_mode="L",
                cbar_location="top",
                cbar_mode="single",
               )

for i in range(4):
    im = grid[i].imshow(toPlot, cmap=cmap, extent = extent_yx)
    #plt.colorbar(im, cax = grid.cbar_axes[0])
grid.cbar_axes[0].colorbar(im)

for cax in grid.cbar_axes:
    cax.toggle_label(False)


In [ ]:
toPlot = 20 * np.log10( np.abs( focusedImg_a[50,:,:].T ) )

fig, ax = plt.subplots(1,1)
im = ax.pcolor(y, z, toPlot, cmap=cmap)
dia.addColorBar( im, ax, aspect="auto" )

#### yx

In [ ]:
toPlot = 20 * np.log10( np.abs( focusedImg_a[:,:,0] ) )

fig, ax = plt.subplots(1,1)
im = ax.imshow(toPlot, origin='lower', cmap=cmap, extent=extent_yx)
dia.addColorBar( im, ax, aspect="equal" )

In [ ]:
opt = disp.OPTt()
opt.db = 1
opt.med_dyn = 100
opt.title = f"Synthesized image: hRail = {z1}, Rx = {Rx}, Tx = {Tx}\nground range, dB  d_gr={dy}, d_az={dx}"
disp.disp_PoSAR_img( focusedImg_a[:,:,0], opt, 0, 0, [yMin,yMax,xMax,xMin] ) # left, right, bottom, top = extent
ax = plt.gca()
ax.invert_xaxis()
ax.set_aspect("equal")
ax.yaxis.set_ticks_position("left")

# Process all

## Define geometry

In [ ]:
hScene = -1

dx = 0.01
dy = 0.01
dz = 0.01
xMin = -0.1
xMax = 0.1
yMin = 0
yMax = 7
zMin = -0.5
zMax = 1.5

imgGeom = (xMin, xMax, dx, yMin, yMax, dy, zMin, zMax, dz)

x = np.arange(xMin, xMax+1e-9, dx)
y = np.arange(yMin, yMax+1e-9, dy)
z = np.arange(zMin, zMax+1e-9, dz)
idx0_x = int( np.size(x)/2 )
idx0_y = int( np.size(y)/2 )
idx0_z = int( np.size(z)/2 )
nbX = x.size
nbY = y.size
nbZ = z.size

# lefg right bottom top
extent_yz = [yMin, yMax, zMin, zMax]
extent_yx = [yMin, yMax, xMin, xMax]

In [ ]:
myParameters           = MyParametersPoSAR_GB()
myParameters.Nx        = nbX
myParameters.Ny        = nbY
myParameters.Nz        = nbZ
myParameters.phi_a_deg = 60 # 10° => 4.3cm

In [ ]:
d_shift = 0.2
overSamplingRatio = 10

img_h = {}

print( f"focusedImg.shape = ({nbX}, {nbY}, {nbZ})")

for data_idx in range(1, 12+1):
    
    path_xxx = base_path + f"/{prefix}{data_idx}"
    z1 = heights[f"{data_idx}"]
    xyz_dic = gb.buildPosUsingTrack( path_xxx + "/track_S" , Dx1, Dx2, h, z1, verbose=1, bizona=biz_offset )
    print( f"z1 = {z1}" )
    
    img_ij = {}
    
    for ij in RxTx:

        Rx, Tx = ij[0], ij[1]
        xyzRx = xyz_dic[ Rx ]
        xyzTx = xyz_dic[ Tx ]
        hAver = ( xyzRx[0,i_z] + xyzTx[0,i_z] ) / 2
    
        read = gb.readData( path_xxx, f"S{ij}.dat" )
        
        sr = np.fft.fftshift( read[i_RD][:,0:read[i_CFG]["Nf"]-1], axes=1 )
        rangeResolution = read[i_CFG]["delta_sr"]
        Naz = sr.shape[0]
        Nf  = sr.shape[1]
    
        Nover   = overSamplingRatio * Nf
        r_base  = np.arange( Nf ) * rangeResolution - d_shift
        r_over  = np.arange( Nover ) * rangeResolution / overSamplingRatio - d_shift
        dr_over = r_over[1] - r_over[0]
    
        myParameters.Nover     = r_over.size
        myParameters.dx        = dr_over
        myParameters.Naz       = Naz
        myParameters.Nf        = Nf
        myParameters.kc        = read[i_CFG]["kc"]
    
        t = time.time()
    
        focusedImg  = np.zeros( (nbX, nbY, nbZ), dtype=complex )
        
        lib.so.backProjectionOmpGroundRange_PoSAR_GB_a( x,
                                                       y,
                                                       z,
                                                       r_over,
                                                       sr.reshape(sr.size),
                                                       xyzRx.reshape(xyzRx.size),
                                                       xyzTx.reshape(xyzTx.size),
                                                       focusedImg.reshape(focusedImg.size),
                                                       myParameters)

        elapsed = time.time() - t
        print( f"hAver = {hAver:.3f}, (Rx,Tx) = ({Rx},{Tx}), execution time = {elapsed:.1f}")
        img_ij[ij] = focusedImg
    img_h[z1] = img_ij

In [ ]:
for idx in heights:
    hRef = heights[idx]
    print(hRef)
    with open(base_path + f"/{date}_hRef{hRef:.3f}_x{xMin:.3f}_{xMax:.3f}.dat", 'wb') as file:
        pk.dump((imgGeom, img_h[hRef]), file)

In [ ]:
np.save(base_path + f"/{date}_h{z1}_{ij}", focusedImg.reshape(nbY,nbX))

In [ ]:
focusedImg.reshape(nbY,nbX).shape

In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot( xyzRx[:, i_x], label=f"x Rx {Rx}" )
ax.plot( xyzRx[:, i_y], label=f"y Rx {Rx}" )
ax.plot( xyzRx[:, i_z], label=f"z Rx {Rx}" )
ax.plot( xyzTx[:, i_x], 'x', label=f"x Tx {Tx}" )
ax.plot( xyzTx[:, i_y], 'x', label=f"y Tx {Tx}" )
ax.plot( xyzTx[:, i_z], 'x', label=f"z Tx {Tx}" )
ax.legend()
ax.grid()

In [ ]:
(2*Dx2+Dx1)/2-1.5

## Plot

In [ ]:
opt = disp.OPTt()
opt.db = 1
opt.med_dyn = 100

## yz

In [ ]:
height = 3.5
Rx = "2"
Tx = "1"
toPlot = 20 * np.log10( np.abs( img_h[height][f"{Rx}{Tx}"][idx0_x,:,:] ) )

opt.title = f"Synthesized image: {height}, Rx = {Rx}, Tx = {Tx}\nground range, dB  d_gr={dy}, d_az={dx}"
disp.disp_PoSAR_img( toPlot, opt, 0, 0, [yMin,yMax,zMin,zMax] ) # left, right, bottom, top = extent
ax = plt.gca()
ax.invert_xaxis()
ax.set_aspect("auto")
ax.yaxis.set_ticks_position("left")

In [ ]:
height = 3.5
Rx = "2"
Tx = "1"
toPlot = 20 * np.log10( np.abs( img_h[height][f"{Rx}{Tx}"][idx0_x,:,:] ) ).T

fig, ax = plt.subplots(1,1)
im = ax.pcolor(y, z, toPlot, cmap='jet')
dia.addColorBar( im, ax, aspect="equal" )

## yx

In [ ]:
height = heights['1']
Rx = "4"
Tx = "3"

In [ ]:
toPlot = img_h[height][f"{Rx}{Tx}"][:,:,idx0_z]

opt.title = f"{date}, h = {height}, Rx = {Rx}, Tx = {Tx}\nground range, dB  d_gr={dy}, d_az={dx}"
disp.disp_PoSAR_img( toPlot, opt, 0, 0, [yMin,yMax,xMax,xMin] ) # left, right, bottom, top = extent
ax = plt.gca()
ax.invert_xaxis()
ax.set_aspect("equal")
ax.xaxis.set_ticks_position("bottom")
ax.yaxis.set_ticks_position("left")
ax.grid()
plt.title( opt.title )

In [ ]:
toPlot = 20 * np.log10( np.abs( img_h[height][f"{Rx}{Tx}"][:,:,idx0_z] ) )

fig, ax = plt.subplots(1,1)
im = ax.pcolormesh(y, x, toPlot, cmap='jet')
dia.addColorBar( im, ax, aspect="equal" )
ax.invert_yaxis()
title = f"{date}, h = {height}, Rx = {Rx}, Tx = {Tx}\nground range, dB  d_gr={dy}, d_az={dx}"
ax.set_title(title)

In [ ]:
fig, ((ax0,ax1),(ax2,ax3)) = plt.subplots(2,2)

IJ = "41"
toPlot = 20 * np.log10( np.abs( img_h[height][IJ][:,:,idx0_z] ) )
im = ax0.pcolormesh(y, x, toPlot, cmap='jet')
dia.addColorBar( im, ax0, aspect="equal" )
ax0.set_title(IJ)

IJ = "43"
toPlot = 20 * np.log10( np.abs( img_h[height][IJ][:,:,idx0_z] ) )
im = ax1.pcolormesh(y, x, toPlot, cmap='jet')
dia.addColorBar( im, ax1, aspect="equal" )
ax1.set_title(IJ)

IJ = "43"
toPlot = 20 * np.log10( np.abs( img_h[height][IJ][:,:,idx0_z] ) )
im = ax2.pcolormesh(y, x, toPlot, cmap='jet')
dia.addColorBar( im, ax2, aspect="equal" )
ax2.set_title(IJ)

IJ = "21"
toPlot = 20 * np.log10( np.abs( img_h[height][IJ][:,:,idx0_z] ) )
im = ax3.pcolormesh(y, x, toPlot, cmap='jet')
dia.addColorBar( im, ax3, aspect="equal" )
ax3.set_title(IJ)

title = f"{date}, h = {height}\nground range, dB  d_gr={dy}, d_az={dx}"
fig.suptitle(title)

## Sum

In [ ]:
imgSum = np.zeros(img_h["171cm"]["21"][x0,:,:].shape, dtype=complex)
imgSumC = np.zeros(img_h["171cm"]["21"][x0,:,:].shape, dtype=complex)
amp = np.zeros(img_h["171cm"]["21"][x0,:,:].shape, dtype=complex)
idxx = x0

for height in heights:
    for ij in RxTx:
        imgSum +=         img_h[height][ij][idxx,:,:]
        amp    += np.abs( img_h[height][ij][idxx,:,:] )

In [ ]:
#imgSum /= amp
#toPlot = 20 * np.log10( np.abs( imgSum.T) )
toPlot = np.abs( imgSum.T)

fig, ax = plt.subplots(1,1)
im = ax.pcolor(y, z, toPlot, cmap='jet')
dia.addColorBar( im, ax, aspect="equal" )

### Monostatic

In [ ]:
imgSum  = np.zeros( (nbY, nbZ), dtype=complex)
imgSumC = np.zeros( (nbY, nbZ), dtype=complex)
amp     = np.zeros( (nbY, nbZ), dtype=complex)
idxx = idx0_x

for height in heights:
    for ij in ["21", "31", "32"]:
        z3 = heights[height]
        imgSum +=         img_h[z3][ij][idxx,:,:]
        amp    += np.abs( img_h[z3][ij][idxx,:,:] )

In [ ]:
imgSumNorm = imgSum / amp
toPlot = np.abs( imgSumNorm.T)

fig, ax = plt.subplots(1,1)
im = ax.pcolormesh(y, z, toPlot, cmap='jet')
dia.addColorBar( im, ax, aspect="equal" )

### Bistatic

In [ ]:
biz_imgSum  = np.zeros( (nbY, nbZ), dtype=complex)
biz_imgSumC = np.zeros( (nbY, nbZ), dtype=complex)
biz_amp     = np.zeros( (nbY, nbZ), dtype=complex)
idxx = idx0_x

for height in heights:
    for ij in ["41", "42", "43"]:
        z3 = heights[height]
        if z3 != 3.5:
            if ij != "41":
                biz_imgSum +=         img_h[z3][ij][idxx,:,:]
                biz_amp    += np.abs( img_h[z3][ij][idxx,:,:] )
        else:
            biz_imgSum +=         img_h[z3][ij][idxx,:,:]
            biz_amp    += np.abs( img_h[z3][ij][idxx,:,:] )

In [ ]:
biz_imgSumNorm = biz_imgSum / biz_amp
toPlot = np.abs( biz_imgSumNorm.T)

fig, ax = plt.subplots(1,1)
im = ax.pcolormesh(y, z, toPlot, cmap='jet')
dia.addColorBar( im, ax, aspect="equal" )